<a href="https://colab.research.google.com/github/revirevy/RAG_NIX/blob/main/RAG_FOR_PDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

chatbot


In [1]:
%pip install -q uv PyPDF2 sentence-transformers langchain faiss-cpu groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.2/16.2 MB 75.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [10]:
!uv pip install -U langchain-community -q --system transformers sentence_transformers numpy PyPDF2  langchain faiss-cpu groq --no-cache-dir


In [3]:
!uv pip install -U -q --system sentence_transformers numpy transformers

In [3]:
!uv pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 -q --system

In [ ]:
!pip show numpy

In [16]:
%pip show sentence_transformers

Name: sentence-transformers
Version: 3.4.1
Summary: State-of-the-Art Text Embeddings
Home-page: https://www.SBERT.net
Author: 
Author-email: Nils Reimers <info@nils-reimers.de>, Tom Aarsen <tom.aarsen@huggingface.co>
License: Apache 2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: huggingface-hub, Pillow, scikit-learn, scipy, torch, tqdm, transformers
Required-by: 


In [1]:
!uv pip show sentence_transformers

Using Python 3.11.11 environment at: /usr
Name: sentence-transformers
Version: 3.4.1
Location: /usr/local/lib/python3.11/dist-packages
Requires: huggingface-hub, pillow, scikit-learn, scipy, torch, tqdm, transformers
Required-by:


In [1]:
import os
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
from groq import Groq
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from tqdm import tqdm.auto as tqdm


In [2]:
def extract_text_from_pdfs(pdf_folder):
    """
    Extracts text from all PDF files in the given folder.
    """
    text_data = ""
    for pdf_file in tqdm(os.listdir(pdf_folder)):
        if pdf_file.endswith('.pdf'):
            reader = PdfReader(os.path.join(pdf_folder, pdf_file))
            for page in reader.pages:
                text_data += page.extract_text()
    return text_data


In [3]:
def split_text_into_chunks(text, chunk_size=500, chunk_overlap=50):
    """
    Splits text into smaller chunks for embedding generation.
    """
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    return splitter.split_text(text)


In [4]:
def create_vector_database(chunks, embedding_model_name="sentence-transformers/all-MiniLM-L6-v2"):
    """
    Creates a vector database from the given chunks using HuggingFace embeddings.
    """
    embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)
    vector_store = FAISS.from_texts(chunks, embeddings)
    return vector_store


In [5]:
def chatbot(query, vector_store, groq_client, model="llama-3.3-70b-versatile"):
    """
    Retrieves relevant chunks from the vector database and generates a response using Groq.
    """
    # Retrieve relevant chunks
    docs = vector_store.similarity_search(query, k=5)
    context = " ".join([doc.page_content for doc in tqdm(docs)])

    # Generate response
    chat_completion = groq_client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": f"Use the following context for your answer: {context}"},
            {"role": "user", "content": query}
        ]
    )
    return chat_completion.choices[0].message.content


In [7]:
def initialize_groq():
    """
    Initializes the Groq client for the chatbot.
    """
    from google.colab import userdata
    xapi_key = userdata.get('GROK')
    # Replace "your_actual_api_key" with your API key
    # client = Groq(api_key="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")
    client = Groq(api_key=xapi_key)

    return client

# Example usage of the chatbot
if __name__ == "__main__":
    # Path to folder containing PDFs
    pdf_folder = "/content/data"  # Update with your folder path containing PDFs

    # Step 1: Extract text from PDFs
    text_data = extract_text_from_pdfs(pdf_folder)

    # Step 2: Split text into chunks
    chunks = split_text_into_chunks(text_data)

    # Step 3: Create a vector database
    vector_store = create_vector_database(chunks)

    # Step 4: Initialize Groq
    groq_client = initialize_groq()  # No arguments passed here

    # Step 5: Query the chatbot
    query = "c'est quoi le PMSI SMR?"
    response = chatbot(query, vector_store, groq_client)

    print("Chatbot Response:", response)


Chatbot Response: Le PMSI SMR fait référence au Programme de Médicalisation des Systèmes d'Information en Santé pour les Soins de Suite et de Réadaptation (SMR). Il s'agit d'un système de collecte et d'analyse de données sur les activités de soins dispensés dans les établissements de santé publics et privés en France.

Le PMSI SMR est un outil de gestion et d'évaluation des soins de suite et de réadaptation, qui vise à améliorer la qualité et l'efficacité des soins dispensés aux patients. Il permet de recueillir et d'analyser des données sur les caractéristiques des patients, les soins dispensés, les durées de séjour, les coûts et les résultats des traitements.

Le PMSI SMR est réglementé par l'arrêté du 23 décembre 2016 modifié, qui définit les cadres et les normes pour la collecte et l'analyse des données. Les établissements de santé publics et privés participant au service public hospitalier sont tenus de participer à ce programme, qui a été mis en place à compter du 1er juillet 199

comment coder en SMR une reeducation pour un amputé de pied

In [8]:
    # Step 5: Query the chatbot
    query = "ccomment coder en SMR une reeducation pour un amputé de pied?"
    response = chatbot(query, vector_store, groq_client)

    print("Chatbot Response:", response)

Chatbot Response: Pour coder en SMR (Système de Médecine de Réadaptation) une rééducation pour un amputé de pied, il faut considérer les différents aspects de la rééducation, tels que l'évaluation, la prise en charge, la rééducation proprement dite, et les éventuels actes de fabrication ou d'adaptation d'appareillage.

Voici quelques étapes pour coder en SMR une rééducation pour un amputé de pied :

1. **Évaluation** : Il faut évaluer les capacités et les besoins de l'amputé de pied. Cela peut inclure des actes tels que :
 * Évaluation de la mobilité et de la stabilité (GKR+213 : Séance collective de rééducation du langage oral, mais il faudrait plutôt utiliser un code spécifique pour l'évaluation de la mobilité)
 * Évaluation de la force et de lacoordination musculaire (BLR+077 : Séance de rééducation oculomotrice, mais il faudrait plutôt utiliser un code spécifique pour l'évaluation de la force et de la coordination musculaire)
2. **Prise en charge** : Il faut prendre en charge l'amp

In [9]:
    # Step 5: Query the chatbot
    query = "ccomment coder en SMR une reeducation pour un amputé de pied en terme de diagnostics : le MMP et le AE?"
    response = chatbot(query, vector_store, groq_client)

    print("Chatbot Response:", response)

Chatbot Response: Pour coder en SMR (Système de Management de la Relation) une rééducation pour un amputé de pied, il est essentiel de choisir les codes diagnostics les plus précis possibles conformément à la Classification Internationale des Maladies (CIM-10) à usage PMSI (Programme de Médicalisation des Systèmes d'Information). Les deux éléments clés à coder sont le MMP (Manifestation Morbide Principale) et l'AE (Affection ou État).

1. **MMP (Manifestation Morbide Principale)**:
   - Le MMP doit refléter la raison principale pour laquelle le patient a été admis ou traité. Dans le cas d'un amputé de pied en rééducation, le MMP pourrait être lié à la complication ou à l'état qui justifie la rééducation. Par exemple, si l'amputation est la conséquence d'une maladie vasculaire, le MMP pourrait être `I70.2` (Artérosclérose des artères des membres inférieurs) ou un code plus spécifique si disponible.
   - Si la rééducation est directement liée à l'amputation elle-même (par exemple, rééduc

In [10]:
# prompt: rajouter juste les deux fonctions pour sauvegarder et charger le vector store

import pickle

def save_vector_store(vector_store, filename="vector_store.pkl"):
    """Saves the vector store to a file using pickle."""
    with open(filename, "wb") as f:
        pickle.dump(vector_store, f)

def load_vector_store(filename="vector_store.pkl"):
    """Loads the vector store from a file using pickle."""
    with open(filename, "rb") as f:
        vector_store = pickle.load(f)
    return vector_store


In [11]:
save_vector_store(vector_store)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')